In [ ]:
# 🧠 TweetCare-Phi3 LoRA Fine-Tuning Notebook

# 📦 Install Required Libraries
!pip install -U transformers datasets peft trl accelerate bitsandbytes

# 📂 Load and Preprocess Dataset
from datasets import load_dataset

# Load a subset of customer support tweets
support_dataset_raw = load_dataset(
    "MohammadOthman/mo-customer-support-tweets-945k", split="train[:1000]"
)

# Reformat data to instruction format
def build_instruction_prompt(record):
    return {
        "text": (
            f"### Instruction:\nRespond to this customer tweet:\n"
            f"### Input:\n{record['input']}\n"
            f"### Response:\n{record['output']}"
        )
    }

formatted_dataset = support_dataset_raw.map(build_instruction_prompt)

# 🧪 Tokenize Dataset
from transformers import AutoTokenizer

model_checkpoint = "microsoft/phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = formatted_dataset.map(
    lambda x: tokenizer(
        x["text"], truncation=True, padding="max_length", max_length=512
    ),
    batched=True
)
tokenized_dataset.set_format("torch")

# 🧠 Load and Configure Base Model with LoRA
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
import torch

quant_config = BitsAndBytesConfig(load_in_4bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

base_model = prepare_model_for_kbit_training(base_model)

lora_settings = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "k_proj", "q_proj", "v_proj", "o_proj",
        "gate_proj", "down_proj", "up_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_with_lora = get_peft_model(base_model, lora_settings)

# 🎯 Set Training Arguments and Start Fine-Tuning
from transformers import TrainingArguments
from trl import SFTTrainer

output_model_dir = "tweetcare-phi3-mini-lora-v1"

training_arguments = TrainingArguments(
    output_dir=output_model_dir,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",
    fp16=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=model_with_lora,
    train_dataset=tokenized_dataset,
    processing_class=tokenizer,
    args=training_arguments
)

trainer.train()

# 💾 Save Fine-Tuned Adapter and Tokenizer
model_with_lora.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

# 🔍 Run Inference
model = model_with_lora
torch.cuda.empty_cache()
model.eval()

prompt = (
    "### Instruction:\n"
    "Respond to this customer tweet:\n"
    "### Input:\n"
    "I was charged twice for my subscription and I want a refund.\n"
    "### Response:\n"
)

inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}

with torch.no_grad():
    generated = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        temperature=0.7,
        use_cache=False,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(generated[0], skip_special_tokens=True)
print(response)

# 📦 Download Fine-Tuned Model (Optional for Colab)
import shutil
shutil.make_archive(output_model_dir, 'zip', output_model_dir)

from google.colab import files
files.download(f"{output_model_dir}.zip")
